In [ ]:
from surianalytics.connectors import RESTSciriusConnector

In [ ]:
from surianalytics.widgets.config import update_connector_time_delta

In [ ]:
CONNECTOR = RESTSciriusConnector()

In [ ]:
CONNECTOR.set_page_size(10000)

In [ ]:
update_connector_time_delta(c=CONNECTOR, dump_path="../config.json")

In [ ]:
from IPython.display import JSON

In [ ]:
RULES = {
    3120001: "http.hostname",
    3120002: "tls.ja3s.hash",
    3120003: "tls.sni",
    3120004: "tls.subject",
    3120005: "tls.serial",
    3120006: "tls.issuerdn",
    3120007: "http.http_user_agent",
    3120009: "ssh.client.software_version",
    3120010: "ssh.server.software_version",
    3120011: "http.server",
    3120012: "http.server",
    3120013: "smb.filename",
    3120014: "smb.filename",
    3120015: "smb.filename",
}

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
import pandas as pd

In [ ]:
DF = pd.DataFrame()
for sid, field in RULES.items():
    resp = (CONNECTOR
    .get_eve_unique_values(qfilter="alert.signature_id: {sid}".format(sid=sid), 
                           field=field,
                           size=10000,
                           counts="no"))
    print(field, len(resp))
    df = pd.DataFrame()
    df["value"] = resp
    df["field"] = field
    df["sid"] = sid
    DF = pd.concat([DF, df], axis=0)

In [ ]:
import ipywidgets as widgets

In [ ]:
from ipywidgets.widgets.interaction import display

In [ ]:
SELECTION = widgets.Combobox(options=[])
TYPES = {}

In [ ]:
def show(limit: int, fields: list):
    pd.set_option('display.max_rows', limit)
    pd.set_option('display.min_rows', limit)
    pd.set_option('display.max_colwidth', None)
    
    df = (
        DF
        .loc[DF["field"].isin(fields)]
        .sort_values(by=["field", "value"])
    )
    d = df.to_dict(orient="records")
    
    global TYPES
    TYPES = {i["value"]: i["field"] for i in d}
    
    global SELECTION
    SELECTION.options = list(df.value.unique())
    
    display(df)
    display(SELECTION)

In [ ]:
widgets.interact(show, 
                 limit=widgets.IntSlider(min=10, max=1000),
                 fields=widgets.SelectMultiple(options=list(DF.field.unique()), 
                                               rows=len(DF.field.unique())))

In [ ]:
SHOW_SIZE = widgets.IntSlider(min=10, max=1000)

In [ ]:
OUTPUT = widgets.Output()
def explore(args):
    OUTPUT.clear_output()
    pd.set_option('display.max_rows', SHOW_SIZE.value)
    pd.set_option('display.min_rows', SHOW_SIZE.value)
    pd.set_option('display.max_colwidth', None)
    
    resp = CONNECTOR.get_events_tail(qfilter="{field}: \"{value}\"".format(value=SELECTION.value, field=TYPES[SELECTION.value]))
    df = pd.json_normalize(resp)
    with OUTPUT:
        
        display(df[[
            "timestamp",
            "event_type",
            "src_ip",
            "dest_ip",
            "flow_id",
        ] + [c for c in ("host",
                         "tls.sni",
                         "alert.signature",
                         "http.hostname",
                         "http.url") if c in list(df.columns.values)]])
        
        display(JSON(resp))
        display(SELECTION.value)

In [ ]:
EXPLORE = widgets.Button(description="Explore")
EXPLORE.on_click(explore)

In [ ]:
SHOW_SIZE

In [ ]:
display(EXPLORE)

In [ ]:
display(OUTPUT)